In [ ]:
%matplotlib inline

En lugar de usar una clase, podemos utilizar una tabla de asociación para 
asegurarnos la relación muchos a muchos

La diferencia frente a usar la clase inscripciones es que podemos acceder directamente de alumnos a materias

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from modelos_muchos_a_muchos_tabla import Alumno, asociacion_alumnos_materias, Materia

# pip install python-environ
import environ
env = environ.Env()
env.read_env(".env")

# URL de conexión a la base de datos SQLite (puedes cambiarlo a tu configuración PostgreSQL)
db_url = 'sqlite:///ejemplo.db'

# Crear una instancia de motor (engine)
engine = create_engine(db_url)

Creación del modelo de datos #
###############################
Crear la tabla (vacía) en la base de datos

In [2]:
from modelos_muchos_a_muchos_tabla import metadata

metadata.tables
metadata.create_all(engine)

# Crear una instancia de sesión
Session = sessionmaker(bind=engine)
session = Session()

agregar registros a la base de datos #
#######################################

In [ ]:
# Crear registros para alumnos
alumno1 = Alumno(nombre='alumno1')
alumno2 = Alumno(nombre='alumno2')
alumno3 = Alumno(nombre='alumno3')
alumno4 = Alumno(nombre='alumno4')

# Crear registros para materias
materia1 = Materia(nombre='Física')
materia2 = Materia(nombre='Química')
materia3 = Materia(nombre='Matemáticas')
materia4 = Materia(nombre='Pintura')


In [ ]:

# Registrar alumnos y materias en la base de datos
session.add_all([alumno1, alumno2, alumno3, alumno4, materia1, materia2, materia3, materia4])
session.commit()



In [ ]:

# Asociar alumnos y materias directamente mediante la tabla de asociación
asociacion1 = asociacion_alumnos_materias.insert().values(alumno_id=alumno1.id, materia_id=materia1.id)
asociacion2 = asociacion_alumnos_materias.insert().values(alumno_id=alumno1.id, materia_id=materia2.id)
asociacion3 = asociacion_alumnos_materias.insert().values(alumno_id=alumno2.id, materia_id=materia2.id)
asociacion4 = asociacion_alumnos_materias.insert().values(alumno_id=alumno3.id, materia_id=materia3.id)


In [ ]:

session.execute(asociacion1)
session.execute(asociacion2)
session.execute(asociacion3)
session.execute(asociacion4)
session.execute(asociacion4)
session.commit()



In [4]:


# Imprimir información sobre las asociaciones
print("alumnos asociados a materias:")
for asociacion in session.query(asociacion_alumnos_materias).all():
    print(f"alumno {asociacion.alumno_id} está asociado a la materia {asociacion.materia_id}")


alumnos asociados a materias:
alumno 1 está asociado a la materia 1
alumno 1 está asociado a la materia 2
alumno 2 está asociado a la materia 2
alumno 3 está asociado a la materia 3
alumno 3 está asociado a la materia 3


In [9]:

materias = session.query(Materia).all()


In [10]:
for materia in materias:
    print(f"En la materia {materia.nombre}")
    if materia.alumnos: 
        # Aquí no usamos inscripciones. Ni mencionamos la tabla asociaciones
        for alumno in materia.alumnos:
            print(f"Tenemos asociado al alumno {alumno.nombre}")
    else:
        print("No hay alumnos asociados")
    


En la materia Física
Tenemos asociado al alumno alumno1
En la materia Química
Tenemos asociado al alumno alumno1
Tenemos asociado al alumno alumno2
En la materia Matemáticas
Tenemos asociado al alumno alumno3
En la materia Pintura
No hay alumnos asociados


In [11]:
materias[1].alumnos[1].nombre


'alumno2'

In [12]:

# Lo mismo ocurre desde alumnos a materias
alumnos = session.query(Alumno).all()
for alumno in alumnos:
    print(f"El alumno {alumno.nombre}")
    if alumno.materias: 
        # Aquí no usamos inscripciones. Ni mencionamos la tabla asociaciones
        for materia in alumno.materias:
            print(f"Está asociado a la materia {materia.nombre}")
    else:
        print("El alumno no tiene materias asociadas")

El alumno alumno1
Está asociado a la materia Física
Está asociado a la materia Química
El alumno alumno2
Está asociado a la materia Química
El alumno alumno3
Está asociado a la materia Matemáticas
El alumno alumno4
El alumno no tiene materias asociadas
